In [1]:
include("framework.jl")
include("create.jl")
using JLD2

mainPath=".."
configPath=mainPath*"/data/config"
outputPath=mainPath*"/data/output"
jlOutputPath=mainPath*"/data/julia"

"../data/julia"

In [7]:
r_i = 1; # 100 mu m radius of ice
r_d = 0.1; # 10 mu m radius of droplets
d = 15; # 1500 mu m distance between hydrometeors

In [5]:
h = 31; # 21 hydrometeors per axis
n = 3; if h%n != 0; error("h%n != 0"); end # 3 ice crystals per axis
h_i = Int(h/n); d_i = n*d # obtain through h, n, d the number of ice crystals and the distance between them



InexactError: InexactError: Int64(10.333333333333334)

(1) CONVERGENCE

In [4]:
# checking which level is required for the ice crystals
for level=1:10
    config = initConfig();
    cubicArray!( 5, d, config, 0, 3, r_d, r_d, r_d, false )
    fAddIco( config, 1, 0.0, 0.0, 0.0, level, r_i, r_i, r_i )
    write_csv(config, configPath*"/convergence_$level.csv")
end

In [5]:
# checking which level is required for the droplets
for level=1:6
    config = initConfig();
    cubicArray!( 5, d, config, 0, level, r_d, r_d, r_d, false )
    fAddIco( config, 1, 0.0, 0.0, 0.0, 9, r_i, r_i, r_i )
    write_csv(config, configPath*"/convergenceArray_$level.csv")
end

In [6]:
# checking which level is required for the droplets if very near to the ice crystals
config = initConfig();
cubicArray!( 5, d, config, 0, 1, r_d, r_d, r_d, false )
fAddIco( config, 1, 0.0, 0.0, 0.0, 8, r_i, r_i, r_i )
write_csv(config, configPath*"/test.csv")

(2) CUBIC ARRAY TEST

In [ ]:
config = initConfig();
seed = 0;  Random.seed!(seed);

cubicArray( 5, 15, config, 0, 1, r_d, r_d, r_d, fAdd=false )

multi_center_ind = ( h_i + 1 ) / 2
center_ind = MultiToLin( multi_center_ind, multi_center_ind, multi_center_ind, h_i )
Theta_i = ones( h_i^3 ); Theta_i[center_ind] = 2
cubicArray!( h_i, d_i, config, Theta_i, 1, r_d, r_d, r_d, false, true )
write_csv(config, configPath*"/test.csv")

(3) CUBIC ARRAY

In [ ]:
a = 1 # 100 mum reference ice crystal radius
aOb = [100.0, 10.0, 6.0, 4.0, 2.4] # a over b

N_i = 1 
N_doN_i = [ 2, 3, 4, 5, 6, 10].^3 # N_d over N_i

h = 31; # 21 hydrometeors per axis
d = 15; # 1500 mu m distance between hydrometeors

In [7]:
multi_center_ind = ( h_i + 1 ) / 2
center_ind = MultiToLin( multi_center_ind, multi_center_ind, multi_center_ind, h_i )
Theta_i = ones( h_i^3 ); Theta_i[center_ind] = 2;

In [4]:
config = initConfig();
seed = 0;  Random.seed!(seed);

#cubicArray( dimension, size, config, theta, resolution, radius1, radius2, radius3, fAdd=false )
#cubicArray!( h, d, config, 0, 1, r_d, r_d, r_d, false )
multi_center_ind = ( h_i + 1 ) / 2
center_ind = MultiToLin( multi_center_ind, multi_center_ind, multi_center_ind, h_i )
Theta_i = ones( h_i^3 ); Theta_i[center_ind] = 2
cubicArray!( h_i, d_i, config, Theta_i, 1, r_d, r_d, r_d, false, true )
write_csv(config, configPath*"/test.csv")

"../data/config/test.csv"

In [24]:
config[getIndex( config, [0,0,0], 50 ), :theta] .= 3;
config[getIndex( config, [0,0,0], 50 ), :]

Row,objectType,theta,positionX,positionY,positionZ,objectParameter1,objectParameter2,objectParameter3,objectParameter4
,String,Int64,Float64,Float64,Float64,Int64,Float64,Float64,Float64
1,icoSphere,3,0.0,0.0,-45.0,1,0.1,0.1,0.1
2,icoSphere,3,0.0,-45.0,0.0,1,0.1,0.1,0.1
3,icoSphere,3,-45.0,0.0,0.0,1,0.1,0.1,0.1
4,icoSphere,3,0.0,0.0,0.0,1,0.1,0.1,0.1
5,icoSphere,3,45.0,0.0,0.0,1,0.1,0.1,0.1
6,icoSphere,3,0.0,45.0,0.0,1,0.1,0.1,0.1
7,icoSphere,3,0.0,0.0,45.0,1,0.1,0.1,0.1


In [ ]:
# function read_data( dataLogFile, dataFile )

simName = "test"

@suppress begin
c, cap = read_data( outputPath, "$simName")
end
df = fuse("$simName", c, cap)
@save "$jlOutputPath/$simName.jld2" df